In [1]:
from pyspark.ml import feature
from pyspark.ml import clustering
from pyspark.ml import Pipeline
from pyspark.sql import functions as fn
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.sql import functions as fn, Row
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql import functions as sf
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.feature import RegexTokenizer
import requests
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import concat, col, lit, lower
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql.functions import concat_ws
from  pyspark.sql.functions import abs
# seting master("local[*]") enables multicore processing on all available logical cores on your machine
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

In [3]:
airlines_df = spark.read.csv(get_training_filename('airlines.csv'), header=True, inferSchema=True)
airports_df = spark.read.csv(get_training_filename('airports.csv'), header=True, inferSchema=True)
flights_df = spark.read.csv(get_training_filename('flights.csv'), header=True, inferSchema=True)

In [4]:
type(flights_df)

Out[5]: pyspark.sql.dataframe.DataFrame

In [5]:
flights_df_sample = flights_df.sample(True, 0.5, 42)

In [6]:
shape = ((flights_df_sample.count(), len(flights_df_sample.columns)))
print('The shape of flights_df_sample:', shape)

The shape of flights_df_sample: (1162809, 31)

In [7]:
flights_df_sample.select([count(when(isnull(c), c)).alias(c) for c in flights_df_sample.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
 0| 0| 0| 0| 0| 0| 7379| 0| 0| 0| 43071| 43071| 44529| 44529| 1| 52456| 52456| 0| 46262| 46262| 0| 46262| 52456| 0| 0| 2862132| 2376073| 2376073| 2376073| 2376073| 2376073|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+

In [8]:
flights_df_sample = flights_df_sample.select('MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_DELAY', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'CANCELLED')

flights_df_sample = flights_df_sample.filter((fn.col('CANCELLED')==0))

In [9]:
flights_df_sample.select("DEPARTURE_DELAY").rdd.max()[0]

Out[20]: 1988

In [10]:
flights_df_sample.select("DEPARTURE_DELAY").rdd.min()[0]

Out[21]: -55

In [11]:
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import udf
from pyspark.sql.types import *

delay_bucketizer = Bucketizer(splits=[ -55, 15, 30, float('Inf') ],inputCol="DEPARTURE_DELAY", outputCol="Delay_Bucket")
flights_df_sample = delay_bucketizer.setHandleInvalid("keep").transform(flights_df_sample)

d = {0.0:"No Delay", 1.0: "Delayed by more than 15mins", 2.0:"Delayed by more than 30 mins"}
udf_foo = udf(lambda x: d[x], StringType())
flights_df_sample = flights_df_sample.withColumn("Flight_Delayed", udf_foo("Delay_Bucket"))

#flights_df_sample = flights_df_sample.withColumn("Flight_Delayed", fn.when(fn.col("DEPARTURE_DELAY")<10, 0).otherwise(1))


bucketizer = Bucketizer(splits=[ 0, 100, 1000, float('Inf') ],inputCol="DISTANCE", outputCol="Distance_Bucket")
flights_df_sample = bucketizer.setHandleInvalid("keep").transform(flights_df_sample)

t = {0.0:"Short", 1.0: "Medium", 2.0:"Long"}
udf_foo = udf(lambda x: t[x], StringType())
flights_df_sample = flights_df_sample.withColumn("Flight_Distance", udf_foo("Distance_Bucket"))

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="AIRLINE", outputCol="Airline_Numeric").fit(flights_df_sample)
flights_df_sample = indexer.transform(flights_df_sample)

from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCol="Airline_Numeric", outputCol="Airline_OHE")
flights_df_sample= encoder.transform(flights_df_sample)

indexer = StringIndexer(inputCol="ORIGIN_AIRPORT", outputCol="OA_Numeric").fit(flights_df_sample)
flights_df_sample = indexer.transform(flights_df_sample)

encoder = OneHotEncoder(inputCol="OA_Numeric", outputCol="Origin_Airport_OHE")
flights_df_sample= encoder.transform(flights_df_sample)

indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="DA_Numeric").fit(flights_df_sample)
flights_df_sample = indexer.transform(flights_df_sample)

encoder = OneHotEncoder(inputCol="DA_Numeric", outputCol="Destination_Airport_OHE")
flights_df_sample= encoder.transform(flights_df_sample)

In [12]:
flights_df_sample.groupBy('Delay_Bucket').agg(fn.count('*')).show()

+------------+--------+
Delay_Bucket|count(1)|
+------------+--------+
 0.0| 2335296|
 1.0| 199534|
 2.0| 327302|
+------------+--------+

In [13]:
flights_df_sample.printSchema()

root
-- MONTH: integer (nullable = true)
-- DAY: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- AIRLINE: string (nullable = true)
-- ORIGIN_AIRPORT: string (nullable = true)
-- DESTINATION_AIRPORT: string (nullable = true)
-- SCHEDULED_DEPARTURE: integer (nullable = true)
-- DEPARTURE_DELAY: integer (nullable = true)
-- DISTANCE: integer (nullable = true)
-- SCHEDULED_ARRIVAL: integer (nullable = true)
-- CANCELLED: integer (nullable = true)
-- Delay_Bucket: double (nullable = true)
-- Flight_Delayed: string (nullable = true)
-- Distance_Bucket: double (nullable = true)
-- Flight_Distance: string (nullable = true)
-- Airline_Numeric: double (nullable = false)
-- Airline_OHE: vector (nullable = true)
-- OA_Numeric: double (nullable = false)
-- Origin_Airport_OHE: vector (nullable = true)
-- DA_Numeric: double (nullable = false)
-- Destination_Airport_OHE: vector (nullable = true)

In [14]:
flights_df_sample.limit(50).toPandas()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,Delay_Bucket,Flight_Delayed,Distance_Bucket,Flight_Distance,Airline_Numeric,Airline_OHE,OA_Numeric,Origin_Airport_OHE,DA_Numeric,Destination_Airport_OHE
0,1,1,4,DL,SFO,MSP,25,-5,1589,602,0,0.0,No Delay,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,1,4,NK,LAS,MSP,25,-6,1299,526,0,0.0,No Delay,2.0,Long,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1,1,4,US,LAX,CLT,30,14,2125,803,0,0.0,No Delay,2.0,Long,8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",14.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1,1,4,DL,LAX,MSP,35,0,1535,609,0,0.0,No Delay,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,1,4,DL,SEA,MSP,40,-1,1399,549,0,0.0,No Delay,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,1,1,4,AS,ANC,SEA,45,-4,1448,509,0,0.0,No Delay,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",66.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,1,1,4,AS,ANC,PDX,50,-4,1542,525,0,0.0,No Delay,2.0,Long,9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",66.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",29.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,1,1,4,DL,PDX,MSP,50,-5,1426,603,0,0.0,No Delay,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",29.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1,1,4,DL,PDX,MSP,50,-5,1426,603,0,0.0,No Delay,2.0,Long,1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",29.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",9.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1,1,4,NK,LAS,MCI,55,25,1139,537,0,0.0,No Delay,2.0,Long,10.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",8.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",36.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [15]:
training_df, testing_df = flights_df_sample.randomSplit([0.9, 0.1])

In [16]:
training_df.groupBy('Delay_Bucket').agg(fn.count('*')).show()

+------------+--------+
Delay_Bucket|count(1)|
+------------+--------+
 0.0| 2101597|
 1.0| 179560|
 2.0| 294507|
+------------+--------+

In [17]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(
    inputCols=["MONTH", "DAY", "DAY_OF_WEEK", "Airline_OHE", "Origin_Airport_OHE", "Destination_Airport_OHE", "SCHEDULED_DEPARTURE", "Distance_Bucket", "SCHEDULED_ARRIVAL", "CANCELLED"], outputCol="features")

In [18]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidatorModel
lr_bce = BinaryClassificationEvaluator(labelCol='Delay_Bucket', metricName='areaUnderROC')
rf_bce = BinaryClassificationEvaluator(labelCol='Delay_Bucket', metricName='areaUnderROC')
gbt_bce = BinaryClassificationEvaluator(labelCol='Delay_Bucket', metricName='areaUnderROC')


from pyspark.ml import evaluation
lr_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Delay_Bucket", metricName="accuracy")
rf_evaluator_accuracy = evaluation.MulticlassClassificationEvaluator(labelCol="Delay_Bucket", metricName="f1")
rf_cv_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Delay_Bucket", metricName="f1")
gbt_evaluator_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Delay_Bucket", metricName="f1")

In [19]:
lr = LogisticRegression(featuresCol='features', labelCol='Delay_Bucket', maxIter=10, regParam=0.3, elasticNetParam=0.8, family='multinomial')
lr_pipeline = Pipeline(stages=[va, lr])
lr_model = lr_pipeline.fit(training_df)
lr_transform = lr_model.transform(testing_df)

In [20]:
predictions_train = lr_transform
y_true = predictions_train.select(['Delay_Bucket']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
 precision recall f1-score support

 0.0 0.82 1.00 0.90 234338
 1.0 0.00 0.00 0.00 19787
 2.0 0.00 0.00 0.00 32558

 micro avg 0.82 0.82 0.82 286683
 macro avg 0.27 0.33 0.30 286683
weighted avg 0.67 0.82 0.74 286683

Out[13]: array([[234338, 0, 0],
 [ 19787, 0, 0],
 [ 32558, 0, 0]])

In [21]:
lr_evaluator_accuracy.evaluate(lr_transform)

Out[14]: 0.8174115660851882

In [22]:
lr_bce.evaluate(lr_transform)

Out[15]: 0.5

In [23]:
rf = RandomForestClassifier(featuresCol='features', labelCol='Delay_Bucket')
rf_pipeline = Pipeline(stages=[va, rf])
rf_model = rf_pipeline.fit(training_df)
rf_transform = rf_model.transform(testing_df) 

In [24]:
predictions_train = rf_transform
y_true = predictions_train.select(['Delay_Bucket']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
/databricks/python/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
 'precision', 'predicted', average, warn_for)
 precision recall f1-score support

 0.0 0.82 1.00 0.90 233233
 1.0 0.00 0.00 0.00 19940
 2.0 0.00 0.00 0.00 32815

 micro avg 0.82 0.82 0.82 285988
 macro avg 0.27 0.33 0.30 285988
weighted avg 0.67 0.82 0.73 285988

Out[22]: array([[233233, 0, 0],
 [ 19940, 0, 0],
 [ 32815, 0, 0]])

In [25]:
rf_evaluator_accuracy.evaluate(rf_transform)

Out[23]: 0.7326725703230782

In [26]:
rf_bce.evaluate(rf_transform)

Out[24]: 0.5574208703479042

In [27]:
randomForest_model = rf_model.stages[-1]
pd.DataFrame(list(zip(flights_df_sample.columns[:21], randomForest_model.featureImportances.toArray())),
             columns = ['column', 'weight']).sort_values('weight', ascending=False)

,column,weight
8,DISTANCE,0.242891
0,MONTH,0.205675
2,DAY_OF_WEEK,0.013553
1,DAY,0.004178
19,DA_Numeric,0.000758
12,Flight_Delayed,0.000000
18,Origin_Airport_OHE,0.000000
17,OA_Numeric,0.000000
16,Airline_OHE,0.000000
15,Airline_Numeric,0.000000


In [28]:
paramGrid = ParamGridBuilder()\
  .addGrid(rf.maxBins, [25, 28, 31])\
  .addGrid(rf.maxDepth, [4, 6, 8])\
  .addGrid(rf.numTrees, [10, 20, 30])\
  .addGrid(rf.impurity, ["entropy", "gini"])\
  .build()

cv = CrossValidator()\
  .setEstimator(rf)\
  .setEvaluator(rf_cv_evaluator_f1)\
  .setEstimatorParamMaps(paramGrid)\
  .setNumFolds(3)

In [29]:
cv_pipeline = Pipeline(stages=[va,cv])
cv_model = cv_pipeline.fit(training_df)

In [30]:
rf_cv_evaluator_f1.evaluate(cv_model.transform(testing_df))

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:468)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:395)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:177)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:169)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:768)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:453)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:234)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:612)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:367)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:782)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:914)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:714)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 11 more

In [31]:
print("Columns: ", rf_transform.select('features').head()[0].size)
print("Rows: ", rf_transform.select('features').count())

Columns: 1274
Rows: 285988

In [32]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

stringIndexer = StringIndexer(inputCol="features", outputCol="stringIndexFeatures")

In [33]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
mlp = MultilayerPerceptronClassifier(featuresCol='features', labelCol='Delay_Bucket', layers=[10,5,4,3])
mlp_pipeline = Pipeline(stages=[va, mlp])
mlp_model = mlp_pipeline.fit(training_df)
mlp_transform = mlp_model.transform(testing_df)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2336216632129506> in <module> 
 2 mlp = MultilayerPerceptronClassifier ( featuresCol = 'features' , labelCol = 'Delay_Bucket' , layers = [ 10 , 5 , 4 , 3 ] ) 
 3 mlp_pipeline = Pipeline ( stages = [ va , mlp ] ) 
 ----> 4 mlp_model = mlp_pipeline . fit ( training_df ) 
 5 mlp_transform = mlp_model . transform ( testing_df ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o8930.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 351.0 failed 1 times, most recent failure: Lost task 0.0 in stage 351.0 (TID 3535, localhost, executor driver): ExecutorLostFailure (executor driver exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 278328 ms
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2349)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2349)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at scala.Option.foreach(Option.scala:257)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskSetFailed(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler

In [34]:
mlp_f1 = evaluation.MulticlassClassificationEvaluator(labelCol="Delay_Bucket", metricName="accuracy")
mlp_f1.evaluate(mlp_transform)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3398089223093774> in <module> 
 1 mlp_f1 = evaluation . MulticlassClassificationEvaluator ( labelCol = "Delay_Bucket" , metricName = "accuracy" ) 
 ----> 2 mlp_f1 . evaluate ( mlp_transform ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 69 return self . copy ( params ) . _evaluate ( dataset ) 
 70 else : 
 ---> 71 return self . _evaluate ( dataset ) 
 72 else : 
 73 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 99 """
 100 self . _transfer_params_to_java ( ) 
 --> 101 return self . _java_obj . evaluate ( dataset . _jdf ) 
 102 
 103 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2548.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 560.0 failed 1 times, most recent failure: Lost task 7.0 in stage 560.0 (TID 5381, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:191)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:62)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:537)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:543)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: A & B Dimension mismatch!
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.ml.ann.BreezeUtil$.dgemm(BreezeUtil.scala:41)
	at org.apache.spark.ml.ann.AffineLayerModel.eval(Layer.scala:164)
	at org.apache.spark.ml.ann.F

In [35]:
predictions_train = mlp_transform
y_true = predictions_train.select(['Delay_Bucket']).collect()
y_pred = predictions_train.select(['prediction']).collect()

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)